In [25]:
import base64, concurrent, cStringIO, glob, json, matplotlib.pyplot, os
import PIL, shutil, StringIO, struct, subprocess, sys, traceback
import numpy as np
from IPython.core.display import HTML

In [58]:
class Color:
    def __init__(self, colorspec_or_array):
        if isinstance(colorspec_or_array, np.ndarray):
            assert(colorspec_or_array.size == 4)
            self._array = colorspec_or_array
        else:
            colorspec = colorspec_or_array
            if colorspec[0] == '#':
                colorspec = colorspec[1:]
            if len(colorspec) == 6:
                colorspec += 'ff'
            assert(len(colorspec) == 8)
            self._array = np.array([
                int(colorspec[0:2], 16),
                int(colorspec[2:4], 16),
                int(colorspec[4:6], 16),
                int(colorspec[6:8], 16)
               ])
        
    def array(self):
        return self._array
    
    def colorspec(self):
        return '#%02x%02x%02x%02x' % (self._array[0], self._array[1], self._array[2], self._array[3])
    
    def float(self):
        return self._array[0] + self._array[1] * 256.0 + self._array[2] * 256.0 * 256.0
    
    def __str__(self):
        return self.colorspec()
    
    def _repr_html_(self):
        html = '<div style="display: inline-block; width: 10px; height: 10px; background-color: %s"></div>' % self.colorspec()
        html += ' ' + self.colorspec()
        return html

class Colormap:
    def __init__(self, colors):
        channels = numpy.vstack([Color(c).array() for c in colors]).transpose()
        linspace = numpy.linspace(0,255,len(colors))
        self.colormap = numpy.vstack([numpy.interp(numpy.arange(0,256), linspace, channels[i]) for i in range(0,4)]).transpose()
        self.colormap = numpy.uint8(self.colormap.round())
        self.colormap = self.colormap.reshape([1,256,4])
        
    def image(self):
        return PIL.Image.fromarray(self.colormap)
        
    def write_image(self, filename):
        self.image().save(filename)
        
    def base64image(self):
        buffer = cStringIO.StringIO()
        self.image().save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue())
    
    def _repr_html_(self):
        b64 = self.base64image()
        return '<img src="data:image/png;base64,%s" style="height:10px; width:256px">' % b64
    
    def __getitem__(self, idx):
        return Color(self.colormap[0][idx])
    
# c=Colormap([
#     '#414141',
#     '#ff0000',
#     '#ffff00',
#     '#00ff00'
# ])
# print(c[255])
# c.write_image('foo.png')

In [ ]:
# write_colormap('earthtime-colormaps/grey-red-yellow-green.png', [
#     '#414141',
#     '#ff0000',
#     '#ffff00',
#     '#00ff00'
# ])
# 
# write_colormap('earthtime-colormaps/grey-green-yellow-red.png', [
#     '#414141',
#     '#00ff00',
#     '#ffff00',
#     '#ff0000'
# ])
# 
# write_colormap('earthtime-colormaps/red-yellow-green_alpha_dd.png', [
#     '#ff0000dd',
#     '#ffff00dd',
#     '#00ff00dd'
# ])
# 
# write_colormap('earthtime-colormaps/red-yellow-green_alpha_cc.png', [
#     '#ff0000cc',
#     '#ffff00cc',
#     '#00ff00cc'
# ])
# 
# write_colormap('earthtime-colormaps/red-yellow-green.png', [
#     '#ff0000',
#     '#ffff00',
#     '#00ff00'
# ])
# 
# write_colormap('earthtime-colormaps/red2-yellow-green2.png', [
#     '#ff0000',
#     '#ff0000',
#     '#ffff00',
#     '#00ff00',
#     '#00ff00'
# ])
# 
# write_colormap('earthtime-colormaps/red-yellow-green5.png', [
#     '#ff0000',
#     '#ffff00',
#     '#00ff00',
#     '#00ff00',
#     '#00ff00',
#     '#00ff00',
#     '#00ff00'
# ])
# 
# write_colormap('earthtime-colormaps/green-yellow-red.png', [
#     '#00ff00',
#     '#ffff00',
#     '#ff0000'
# ])
# 
# write_colormap('earthtime-colormaps/green-yellow-red-alpha_dd.png', [
#     '#00ff00dd',
#     '#ffff00dd',
#     '#ff0000dd'
# ])
# 
# write_colormap('earthtime-colormaps/green-yellow-red-alpha_cc.png', [
#     '#00ff00cc',
#     '#ffff00cc',
#     '#ff0000cc'
# ])
# 
# write_colormap('earthtime-colormaps/green-yellow-red5.png', [
#     '#00ff00',
#     '#ffff00',
#     '#ff0000',
#     '#ff0000',
#     '#ff0000',
#     '#ff0000',
#     '#ff0000'
# ])
# write_colormap('earthtime-colormaps/white-red.png', [
#     '#ffffff',
#     '#ff0000'
# ])
# write_colormap('earthtime-colormaps/white-red-dkred.png', [
#     '#ffffff',
#     '#ff0000',
#     '#8b0000'
# ])
# write_colormap('earthtime-colormaps/pink-red-dkred.png', [
#     '#ffc0cb',
#     '#ff0000',
#     '#8b0000'
# ])
# 
# write_colormap('earthtime-colormaps/lightyellow-orange-deeppink-darkred.png', [
#     '#ffffe0', '#ffa474', '#db4551', '#8b0000'
# ])
# 
# write_colormap('earthtime-colormaps/grey-blue.png', [
#     '#d2c6d3', '#000cff'
# ])
# 
# write_colormap('earthtime-colormaps/dkgrey-magenta.png', [
#     '#414141', '#ff2eff'
# ])
# 
# write_colormap('earthtime-colormaps/red-blue.png', [
#     '#ff0000', '#0000ff'
# ])
# 
# write_colormap('earthtime-colormaps/blue-red.png', [
#     '#0000ff', '#ff0000'
# ])
# 
# write_colormap('earthtime-colormaps/yellow-red.png', [
#     '#eeee00', '#ff0000'
# ])
# 
# write_colormap('earthtime-colormaps/red-yellow.png', [
#     '#ff0000', '#ffff00'
# ])
# 
# write_colormap('earthtime-colormaps/yellow-green.png', [
#     '#eeee00', '#00ff00'
# ])
# 
# write_colormap('earthtime-colormaps/green-yellow.png', [
#     '#00ff00', '#ffff00'
# ])
# 
# write_colormap('earthtime-colormaps/red-yellow-green-purple.png', [
#     '#ff0000',
#     '#ffff00',
#     '#00ff00',
#     '#ff00ff'
# ])
# 
# write_colormap('earthtime-colormaps/grey-red-yellow-green-purple.png', [
#     '#414141',
#     '#ff0000',
#     '#ffff00',
#     '#00ff00',
#     '#ff00ff'
# ])